In [1]:
import pandas as pd
from html import unescape
import re
import numpy as np

from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

import pickle

In [2]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [3]:
data

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, with, yoyou, all, t...","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"
...,...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory: left-right polarisation! #tru...,thought factory leftright polarisation trump u...,"[thought, factory, leftright, polarisation, tr...","[thought, factory, leftright, polarisation, tr...","[thought, factori, leftright, polaris, trump, ...","[think, factory, leftright, polarisation, trum..."
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverready, fo..."
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...,hillary campaigned today in ohioomg amp used w...,"[hillary, campaigned, today, in, ohioomg, amp,...","[hillary, campaigned, today, ohioomg, amp, use...","[hillari, campaign, today, ohioomg, amp, use, ...","[hillary, campaign, today, ohioomg, amp, use, ..."
49157,49158,NaN,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, work, confer, right, mindset, lead, cu...","[happy, work, conference, right, mindset, lead..."


**1. Создайте мешок слов с помощью `sklearn.feature_extraction.text.CountVectorizer.fit_transform()`. Применим его к `'tweet_stemmed'` и `'tweet_lemmatized'` отдельно.**

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью `max_df`.

●	Ограничим количество слов, попадающий в мешок, с помощью `max_features` = 1000.

●	Исключим стоп-слова с помощью `stop_words='english'`.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью `CountVectorizer.get_feature_names()`.


In [4]:
corpus_stemmed = [','.join(i) for i in data['tweet_stemmed']]
count_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_stemmed = count_vectorizer.fit_transform(corpus_stemmed)

In [5]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
corpus_lemmatized = [','.join(i) for i in data['tweet_lemmatized']]
count_vectorizer = CountVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_lemmatized = count_vectorizer.fit_transform(corpus_lemmatized)

In [7]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49155,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49156,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49157,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**2. Создайте мешок слов с помощью `sklearn.feature_extraction.text.TfidfVectorizer.fit_transform()`. Применим его к `'tweet_stemmed'` и `'tweet_lemmatized'` отдельно.**

●	Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью `max_df`.

●	Ограничим количество слов, попадающий в мешок, с помощью `max_features` = 1000.

●	Исключим стоп-слова с помощью `stop_words='english'`.

●	Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().


In [8]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_stemmed = tfidf_vectorizer.fit_transform(corpus_stemmed)

In [9]:
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yo,yoga,york,young,youtub,yoyou,yoyour,yoyoy,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.743324,0.391965,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [10]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word', binary=False, max_df=0.9, max_features = 1000, stop_words='english')
bag_of_words_lemmatized = tfidf_vectorizer.fit_transform(corpus_lemmatized)

In [11]:
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words_lemmatized.toarray(), columns = feature_names)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yoga,york,young,youtube,yoyou,yoyour,yoyoyou,yr,yrs,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.742601,0.391988,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
49157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0


**3. Натренируем gensim.models.Word2Vec модель на наших данных.**

●	Тренировать будем на токенизированных твитах `combine_df['tweet_token']`

●	Установим следующие параметры: `size=200`, `window=5`, `min_count=2`, `sg = 1`, `hs = 0`, `negative = 10`, `workers= 32`, `seed = 34`.

●	Используем функцию `train()` с параметром `total_examples` равным длине `combine_df['tweet_token']`, количество epochs установим 20.


In [12]:
sentences = [eval(str(item)) for item in data['tweet_token']]

In [13]:
sentences

[['when',
  'father',
  'is',
  'dysfunctional',
  'and',
  'is',
  'so',
  'selfish',
  'he',
  'drags',
  'his',
  'kids',
  'into',
  'his',
  'dysfunction',
  'run'],
 ['thanks',
  'for',
  'lyft',
  'credit',
  'can',
  'not',
  'use',
  'cause',
  'they',
  'do',
  'not',
  'offer',
  'wheelchair',
  'vans',
  'in',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'your', 'majesty'],
 ['model',
  'love',
  'yoyou',
  'take',
  'with',
  'yoyou',
  'all',
  'the',
  'time',
  'in',
  'yoyour'],
 ['factsguide', 'society', 'now', 'motivation'],
 ['huge',
  'fan',
  'fare',
  'and',
  'big',
  'talking',
  'before',
  'they',
  'leave',
  'chaos',
  'and',
  'pay',
  'disputes',
  'when',
  'they',
  'get',
  'there',
  'allshowandnogo'],
 ['camping', 'tomorrow', 'danny'],
 ['the',
  'next',
  'school',
  'year',
  'is',
  'the',
  'year',
  'for',
  'exams',
  'can',
  'not',
  'think',
  'about',
  'that',
  'school',
  'exams',
  'hate',
  'imagine',
  'actorslife',
  'revol

In [14]:
model = gensim.models.Word2Vec(sentences, size=200, window=5, min_count=2, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)

In [15]:
model.train(sentences, total_examples=data['tweet_token'].shape[0], epochs=20)

(9048325, 11671440)

**4. Давайте немного потестируем нашу модель `Word2Vec` и посмотрим, как она работает. Мы зададим слово `positive = "dinner"`, и модель вытащит из корпуса наиболее похожие слова c помощью функции `most_similar`. То же самое попробуем со словом "trump".**

In [16]:
model.most_similar('dinner')

<ipython-input-16-7572d32f55f9>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('dinner')


[('bihdaydinner', 0.5522367358207703),
 ('tacotuesday', 0.5403922200202942),
 ('lastnight', 0.53597491979599),
 ('shawarma', 0.5343489050865173),
 ('hamburger', 0.5318780541419983),
 ('photoftheday', 0.5299574136734009),
 ('bolognese', 0.5296697616577148),
 ('cookout', 0.5264283418655396),
 ('sissy', 0.5261825919151306),
 ('spaghetti', 0.5216397643089294)]

In [17]:
model.most_similar('trump')

<ipython-input-17-d253bc5c5242>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('trump')


[('donald', 0.5839777588844299),
 ('suppoer', 0.5297306776046753),
 ('conman', 0.5124558806419373),
 ('fuhered', 0.5097458362579346),
 ('dumptrump', 0.5028122067451477),
 ('unfavorability', 0.5011065602302551),
 ('wonthey', 0.4975353479385376),
 ('potus', 0.49647822976112366),
 ('carl', 0.49244922399520874),
 ('unstable', 0.49143746495246887)]

**5. Из приведенных выше примеров мы видим, что наша модель `word2vec` хорошо справляется с поиском наиболее похожих слов для данного слова. Но как она это делает? Она изучила векторы для каждого уникального слова наших данных и использует косинусное сходство, чтобы найти наиболее похожие векторы (слова).**

Давайте проверим векторное представление любого слова из нашего корпуса, например `"food"`.

In [18]:
vector = model.wv['food']

In [19]:
vector

array([-7.81212747e-02, -3.52687031e-01, -5.71428835e-01, -3.14780146e-01,
        2.45003983e-01, -3.43355723e-02,  4.57552046e-01,  5.18384039e-01,
        4.26179111e-01,  5.29876590e-01, -5.30844271e-01,  6.58440441e-02,
       -1.16182518e+00,  2.24438936e-01,  4.48027968e-01, -2.41893589e-01,
        2.90929489e-02,  3.16414945e-02,  1.57585427e-01, -3.11908811e-01,
       -2.07383156e-01, -2.25203827e-01,  2.06614077e-01,  5.86982548e-01,
        7.89572179e-01,  5.79739809e-01,  9.86346483e-01, -5.21738768e-01,
        1.24661013e-01,  8.42857659e-01, -1.49582282e-01, -3.07144783e-02,
       -3.00371349e-01, -5.57171643e-01,  2.57489979e-01,  2.99271494e-01,
        5.34022450e-01,  6.72077894e-01, -6.33804321e-01, -6.45745814e-01,
       -3.63917530e-01, -1.50884986e-01,  3.23637784e-01, -1.90808639e-01,
       -3.01981360e-01,  1.50534198e-01,  1.07173949e-01,  1.20873022e+00,
        5.16056657e-01, -4.38588530e-01,  2.67089218e-01,  8.94542411e-02,
       -8.45925435e-02,  

**6. Поскольку наши данные содержат твиты, а не только слова, нам придется придумать способ использовать векторы слов из модели word2vec для создания векторного представления всего твита.**

**Существует простое решение этой проблемы, мы можем просто взять среднее значение всех векторов слов, присутствующих в твите. Длина результирующего вектора будет одинаковой, то есть 200. Мы повторим тот же процесс для всех твитов в наших данных и получим их векторы. Теперь у нас есть 200 функций word2vec для наших данных.**

**Необходимо создать вектор для каждого твита, взяв среднее значение векторов слов, присутствующих в твите. В цикле сделать:  vec += model_w2v[word].reshape((1, size)) и поделить финальный вектор на количество слов в твите.**

**На выходе должен получиться wordvec_df.shape = (49159, 200).**

In [20]:
vecs = []
vec = 0
for sentence in sentences:
    for word in sentences:
        vec += model.wv[word]
    vec = vec/len(sentence)
    vecs.append(vec)

KeyError: "word 'disapointed' not in vocabulary"

In [34]:
sentences

[['when',
  'father',
  'is',
  'dysfunctional',
  'and',
  'is',
  'so',
  'selfish',
  'he',
  'drags',
  'his',
  'kids',
  'into',
  'his',
  'dysfunction',
  'run'],
 ['thanks',
  'for',
  'lyft',
  'credit',
  'can',
  'not',
  'use',
  'cause',
  'they',
  'do',
  'not',
  'offer',
  'wheelchair',
  'vans',
  'in',
  'pdx',
  'disapointed',
  'getthanked'],
 ['bihday', 'your', 'majesty'],
 ['model',
  'love',
  'yoyou',
  'take',
  'with',
  'yoyou',
  'all',
  'the',
  'time',
  'in',
  'yoyour'],
 ['factsguide', 'society', 'now', 'motivation'],
 ['huge',
  'fan',
  'fare',
  'and',
  'big',
  'talking',
  'before',
  'they',
  'leave',
  'chaos',
  'and',
  'pay',
  'disputes',
  'when',
  'they',
  'get',
  'there',
  'allshowandnogo'],
 ['camping', 'tomorrow', 'danny'],
 ['the',
  'next',
  'school',
  'year',
  'is',
  'the',
  'year',
  'for',
  'exams',
  'can',
  'not',
  'think',
  'about',
  'that',
  'school',
  'exams',
  'hate',
  'imagine',
  'actorslife',
  'revol

In [33]:
model.wv.vocab

{'when': <gensim.models.keyedvectors.Vocab at 0x20234d66d00>,
 'father': <gensim.models.keyedvectors.Vocab at 0x20234d66d30>,
 'is': <gensim.models.keyedvectors.Vocab at 0x20234d66d90>,
 'dysfunctional': <gensim.models.keyedvectors.Vocab at 0x20234d667f0>,
 'and': <gensim.models.keyedvectors.Vocab at 0x20234d66e20>,
 'so': <gensim.models.keyedvectors.Vocab at 0x20234d66e80>,
 'selfish': <gensim.models.keyedvectors.Vocab at 0x20234d66910>,
 'he': <gensim.models.keyedvectors.Vocab at 0x20234d66f10>,
 'drags': <gensim.models.keyedvectors.Vocab at 0x20234d66f70>,
 'his': <gensim.models.keyedvectors.Vocab at 0x20234d66fd0>,
 'kids': <gensim.models.keyedvectors.Vocab at 0x20234d661f0>,
 'into': <gensim.models.keyedvectors.Vocab at 0x20234d6e040>,
 'dysfunction': <gensim.models.keyedvectors.Vocab at 0x20234d6e0a0>,
 'run': <gensim.models.keyedvectors.Vocab at 0x20234d6e100>,
 'thanks': <gensim.models.keyedvectors.Vocab at 0x20234d6e160>,
 'for': <gensim.models.keyedvectors.Vocab at 0x20234d6e

Не понял почему `disapointed` алгоритм "выкинул" из словаря